In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import pandas as pd
import tensorflow as tf
import shutil
import numpy as np

tf.logging.set_verbosity(tf.logging.INFO)



In [ ]:
feed_data8=pd.read_pickle("feed_data8.pickle")

In [ ]:
feed_data8=feed_data8.drop(columns=["english", "news_id", "crtd_date", "pv3hr", "pv3day"])

In [ ]:
feed_data8.head()

In [ ]:
feed_data8.columns

In [ ]:
df_train=feed_data8.iloc[:70000,:]
df_valid=feed_data8.iloc[70000:,:]

In [ ]:
df_train.to_csv("./final_data2/df_train.csv", index=False)
df_valid.to_csv("./final_data2/df_valid.csv", index=False)

In [ ]:
df_valid=pd.read_csv("./final_data2/df_valid.csv")

In [ ]:
len(df_valid)

In [ ]:
# fdata_shuffled=feed_data8.sample(frac=1).reset_index(drop= True)

In [ ]:
# fdata_shuffled.head(100)

In [ ]:

# df_train=fdata_shuffled.iloc[:70000,:]
# df_valid=fdata_shuffled.iloc[70000:,:]


In [ ]:
#alternate split

In [ ]:
# df_train.to_csv("./final_data/df_train.csv", index=False)
# df_valid.to_csv("./final_data/df_valid.csv", index=False)

In [ ]:
CSV_COLUMNS=['upv3day', 'polarity', 'subjectivity', 'num_of_sentences',
       'length', 'num_of_identities', 'other_entities', 'upv3hr', 'msaav10', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14',
       'cat', 'weekday', 'hour']
FEATURES = CSV_COLUMNS[1:]
LABEL = CSV_COLUMNS[0]
#deFAULTS DEFINES datatype of column in tf.decode_csv
DEFAULTS = [[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],["none"],[0],[0]]



In [ ]:
def read_dataset(filename, mode, batch_size = 500):
   def _input_fn():
      def decode_csv(row):
        columns = tf.decode_csv(row, record_defaults = DEFAULTS)
        features = dict(zip(CSV_COLUMNS, columns))
        label = features.pop(LABEL) # remove label from features and store
        return features, label

      # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
      filenames_dataset = tf.data.Dataset.list_files(filename, shuffle=False)
      # Read lines from text files
      textlines_dataset = filenames_dataset.flat_map(tf.data.TextLineDataset).skip(count=1)
      # Parse text lines as comma-separated values (CSV)
      dataset = textlines_dataset.map(decode_csv)

      # Note:
      # use tf.data.Dataset.flat_map to apply one to many transformations (here: filename -> text lines)
      # use tf.data.Dataset.map      to apply one to one  transformations (here: text line -> feature list)

      if mode == tf.estimator.ModeKeys.TRAIN:
          num_epochs = None # loop indefinitely, so limit will be difined by max steps in estimator
          dataset = dataset.shuffle(buffer_size = 70000, seed=2)#100 * batch_size
      else:
          num_epochs = 1 # end-of-input after this

      dataset = dataset.repeat(num_epochs).batch(batch_size)

      return  dataset.make_one_shot_iterator().get_next()
   return _input_fn




def get_train_input_fn():
  return read_dataset('./final_data2/df_train.csv', mode = tf.estimator.ModeKeys.TRAIN, batch_size=70000)

def get_valid_input_fn():
  return read_dataset('./final_data2/df_valid.csv', mode = tf.estimator.ModeKeys.EVAL,batch_size=5000 )

def make_feature_cols():#no y here , must enclose cat columns in indentity or embedding cols for dnn
    categorical_column1=tf.feature_column.categorical_column_with_vocabulary_list(key='cat', vocabulary_list=('blogs','business','dharm','education-and-jobs','entertainment','international','life-style','nari','national','other','regional','sports'))
    categorical_column2=tf.feature_column.categorical_column_with_identity(key='weekday', num_buckets=7)
    categorical_column3=tf.feature_column.bucketized_column(source_column=tf.feature_column.numeric_column("hour"), boundaries=[6,12,16,20])
    input_columns = [tf.feature_column.numeric_column('polarity'),
                     tf.feature_column.numeric_column('subjectivity'),
                     tf.feature_column.numeric_column('num_of_sentences'),
                     tf.feature_column.numeric_column('length'),
                     tf.feature_column.numeric_column('num_of_identities'),
                     tf.feature_column.numeric_column('other_entities'),
                     tf.feature_column.numeric_column('upv3hr'),
                     tf.feature_column.numeric_column('msaav10'),
                     tf.feature_column.numeric_column('0'),
                     tf.feature_column.numeric_column('1'),
                     tf.feature_column.numeric_column('2'),
                     tf.feature_column.numeric_column('3'),
                     tf.feature_column.numeric_column('4'),
                     tf.feature_column.numeric_column('5'),
                     tf.feature_column.numeric_column('6'),
                     tf.feature_column.numeric_column('7'),
                     tf.feature_column.numeric_column('8'),
                     tf.feature_column.numeric_column('9'),
                     tf.feature_column.numeric_column('10'),
                     tf.feature_column.numeric_column('11'),
                     tf.feature_column.numeric_column('12'),
                     tf.feature_column.numeric_column('13'),
                     tf.feature_column.numeric_column('14'),
                     tf.feature_column.embedding_column(categorical_column1,dimension=10),
                     
                     tf.feature_column.indicator_column(categorical_column2),
                     tf.feature_column.indicator_column(categorical_column3)]
    return input_columns

def serving_input_fn():#actual datatype in data for serving
    feature_placeholders =  {
        column.name: tf.placeholder(tf.float32, [None]) for column in list(make_feature_cols())[:23]
    }

    feature_placeholders["cat"]=tf.placeholder(tf.string, [None])
    feature_placeholders["weekday"]=tf.placeholder(tf.int32, [None])
    feature_placeholders["hour"]=tf.placeholder(tf.int32, [None])
    
    
    features = feature_placeholders
    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)


def my_metric(labels, predictions):
    pred_values = predictions['predictions']
    return {'metric1': tf.metrics.root_mean_squared_error(labels, pred_values)}

run_config = tf.estimator.RunConfig( save_checkpoints_secs = 5, keep_checkpoint_max = 4, tf_random_seed=100)


def train_and_evaluate(num_train_steps):
    #tf.summary.FileWriterCache.clear() # ensure filewriter cache is clear for TensorBoard events file
    estimator = tf.estimator.DNNRegressor(
        config = run_config,
        model_dir = "./cloudmlemodels",
        feature_columns = make_feature_cols(),
        hidden_units = [100,90,81,73],
        optimizer=tf.train.AdagradOptimizer(learning_rate=0.005))
    estimator = tf.contrib.estimator.add_metrics(estimator, my_metric)
    train_spec = tf.estimator.TrainSpec(
        input_fn =get_train_input_fn(),
         max_steps = num_train_steps)
    exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
    eval_spec = tf.estimator.EvalSpec(
        input_fn = get_valid_input_fn(),
        steps = None,#none will evaluate whole eval data in each test
        start_delay_secs = 10,
        throttle_secs = 10,
        exporters = exporter)
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)




In [ ]:
tf.feature_column.numeric_column('msaav10').name

In [ ]:
dict(df_train.iloc[500,:])

In [ ]:
OUTDIR = "./cloudmlemodels"


# Run training    
#shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
#tf.summary.FileWriterCache.clear() # ensure filewriter cache is clear for TensorBoard events file
train_and_evaluate(80)




In [ ]:

model=tf.estimator.DNNRegressor(
        config = run_config,
         model_dir = "./cloudmlemodels",
         feature_columns = make_feature_cols(),
         hidden_units = [100,90,81,73])

In [ ]:
model.train(input_fn =get_valid_input_fn)

In [ ]:
def print_rmse(model, df):
  metrics = model.evaluate(input_fn = get_valid_input_fn(df))
  print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))



In [ ]:
model = tf.contrib.estimator.add_metrics(model, my_metric)


In [ ]:
model.evaluate(input_fn = get_valid_input_fn())

In [ ]:
t_pred=list(model.predict(input_fn = get_valid_input_fn()))

In [ ]:
predictions = model.predict(input_fn = get_valid_input_fn())


pred=list(predictions)
#pred={key:value for key,value in enumerate(pred)}
len(pred)
